In [3]:
# !mkdir model # uncomment if directory doesnt exist

In [5]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q5_K_M.gguf --local-dir .\model --local-dir-use-symlinks False

.\model\mistral-7b-instruct-v0.2.Q5_K_M.gguf


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf to C:\Users\ngkey\.cache\huggingface\hub\tmp7kp2ehrb


In [9]:
#python -m llama_cpp.server --model .\model\mistral-7b-instruct-v0.2.Q5_K_M.gguf
# paste into terminal, will run continuously

In [4]:
!pip install pandas

In [1]:
import csv, json, warnings
import pandas as pd

In [2]:

data = pd.read_csv('./data/final_data.csv')

In [3]:
data.head()

,date,client,bank,rating,review,developer_response
0,2024-03-06 08:36:37,Apple,GXS,4.0,Great banking app with attractive interest rat...,"{'id': 42449861, 'body': ""Hi Eugene! We're thr..."
1,2022-11-09 03:13:16,Apple,GXS,5.0,"A bank like no other, no bank have such amazin...","{'id': 33087200, 'body': ""Hi Dfdsa132! We're d..."
2,2023-09-09 08:33:43,Apple,GXS,1.0,Notice that the drop in interest rate of 0.8% ...,"{'id': 38832230, 'body': 'Hi there! We’re sorr..."
3,2023-06-27 04:01:39,Apple,GXS,2.0,Sending money into my GXS account is a breeze ...,"{'id': 37376460, 'body': 'Hi Kenny, thank you ..."
4,2022-10-02 04:17:49,Apple,GXS,5.0,I have to say that the UI/UX is one of the bes...,"{'id': 32373818, 'body': 'Hi Mephosis, we are ..."


In [4]:
data[['bank']].drop_duplicates()

,bank
0,GXS
132,Trust
675,MariBank


In [20]:
data["id"] = data.index # add in review indexing for matching later
overall_reviews = data[['id', 'bank', 'review']]
gxs = overall_reviews.query('bank == "GXS"')
trust = overall_reviews.query('bank == "Trust"')
maribank = overall_reviews.query('bank == "MariBank"')
data.groupby("bank").count()

,date,client,rating,review,developer_response,id
bank,,,,,,
GXS,331,331,331,331,327,331
MariBank,21,21,21,21,20,21
Trust,742,742,742,742,673,742


In [9]:
max_len = -1
index = 0
for ele in data["review"]:
    if len(ele) > max_len:
        max_len = len(ele)
        res = ele
        max_index = index
    index +=1
print(max_index, max_len, res)

135 1235 The interface is not intuitive at all. You literally have to keep tap this and that to get between pages. It's infuriating to say the least. Not to mention, whoever is running this credit card is probably the most apathetic person on Earth. Literally makes you feel like you owe them. Late payment? Good luck. You got to plead your case. Asked for a waiver but you need to provide reason. Are we in school? The cs makes you feel like they are doing you a service and you should be grateful. "On a one-time goodwill" I quote them. There's nothing good about this card other than the zero foreign transaction fees. Be careful not to miss your repayment date and think before you set it cause you cannot change it. Very incredibly inflexible to say the least. The cs increases your blood pressure. Even telcos have better cs. Sign for the freebies, redeem those and run. Please get this right, customers do not owe you our business. Stop giving us your elitist attitude. It really feels like I 

In [10]:
data[data.review == res]

,date,client,bank,rating,review,developer_response,id
135,2023-08-22 15:41:09,Apple,Trust,1.0,The interface is not intuitive at all. You lit...,"{'id': 38613851, 'body': ""Hi sushiica, we are ...",136


In [11]:
from llama_cpp import Llama

In [12]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    # Your problematic instruction(s) here
    model = Llama(
        model_path="./model/mistral-7b-instruct-v0.2.Q5_K_M.gguf",
        n_gpu_layers=-1, # Uncomment to use GPU acceleration
        # seed=1337, # Uncomment to set a specific seed
        n_ctx=2048, # Uncomment to increase the context window
        chat_format="chatml"
    )

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./model/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:          

In [14]:
# print(json.dumps(model(
#       "Q: Name the planets in the solar system? A: ", # Prompt
#       max_tokens=32, # Generate up to 32 tokens, set to None to generate up to the end of the context window
#       stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
#       echo=True # Echo the prompt back in the output
# )))

In [15]:
system_prompt = f"""You are a helpful assistant to a customer experience team that outputs in JSON. Referring to the list of mobile app reviews you will be provided, perform the following tasks: 
        1. Summarise each review.
        2. Identify the sentiment of each review of an app. That is to say, conclude whether it is Positive, Negative, or Neutral. 
"""
synthetic_reviews = f"""
0. "This app has revolutionized the way I manage my finances! It's intuitive, user-friendly, and offers a wide range of features like budget tracking and bill payments. I especially love the quick and secure transaction processing."
1. "I've tried several banking apps before, but this one stands out. It's incredibly convenient, with seamless integration across devices. Plus, the customer service is top-notch—I had an issue with a transaction, and they resolved it promptly."
2. "Five stars all the way! This app has made banking so much simpler. I can deposit checks, transfer money, and even apply for loans—all from the comfort of my couch. It's like having a bank branch in my pocket!"
3. "Decent app, but there's room for improvement. It gets the job done for basic transactions, but the interface feels a bit outdated. Also, I've encountered a few glitches while navigating certain features."
4. "It's an okay app. I appreciate the convenience it offers, but it could use some enhancements in terms of personalization. Customizable notifications and more detailed spending insights would be nice additions."
5. "Works fine most of the time, but it can be slow to load, especially during peak hours. Also, I wish they offered more incentives for using the app regularly, like cashback rewards or discounts at partner stores."
6. "Absolutely terrible. I've had nothing but problems with this app—from unauthorized charges to frequent crashes. It's unreliable and frustrating to use. I'm considering switching to a different bank altogether."
7. "Stay away from this app if you value your sanity. It's riddled with bugs and glitches that never seem to get fixed. Plus, the customer support is virtually nonexistent—I've been waiting for a response to my complaint for weeks."
8. "I can't believe I trusted this app with my money. Not only is it unreliable, but it's also a security risk. I've had my account hacked twice in the past year, and the bank's response was inadequate. Do yourself a favor and find a more trustworthy banking solution."
"""
#history[0]["content"] = system_prompt
#history[1]["content"] = synthetic_reviews
history = [
    {"role": "system", "content": system_prompt},
]

In [13]:
# these chunks try to get sentiment only, but do not work. this LLM likes to be longwinded
system_prompt_sent_only = f"""You are a helpful assistant to a customer experience team. Referring to the list of mobile app reviews you will be provided, identify the sentiment of each review of an app. That is to say, conclude whether it is Positive, Negative, or Neutral. 
ONLY return the sentiment tagging. NOTHING ELSE
"""
history_sent_only = [
    {"role": "system", "content": system_prompt},
    {"role": "system", "content": synthetic_reviews}
]
completion_sent_only = model.create_chat_completion(
    # model="local-model", # this field is currently unused
    messages=history_sent_only,
    temperature=0.7,
    # stream=True,
)
print(completion_sent_only)
print(completion_sent_only['choices'][0]['message']['content'])

NameError: name 'system_prompt' is not defined

In [16]:
split_reviews = synthetic_reviews.splitlines()[1:]
print(split_reviews)
synth_review_list = []
for s in split_reviews:
    temp = {"role": "user", "content": s}
    synth_review_list.append(temp)

['0. "This app has revolutionized the way I manage my finances! It\'s intuitive, user-friendly, and offers a wide range of features like budget tracking and bill payments. I especially love the quick and secure transaction processing."', '1. "I\'ve tried several banking apps before, but this one stands out. It\'s incredibly convenient, with seamless integration across devices. Plus, the customer service is top-notch—I had an issue with a transaction, and they resolved it promptly."', '2. "Five stars all the way! This app has made banking so much simpler. I can deposit checks, transfer money, and even apply for loans—all from the comfort of my couch. It\'s like having a bank branch in my pocket!"', '3. "Decent app, but there\'s room for improvement. It gets the job done for basic transactions, but the interface feels a bit outdated. Also, I\'ve encountered a few glitches while navigating certain features."', '4. "It\'s an okay app. I appreciate the convenience it offers, but it could us

In [17]:
# test run on synthetic data
answers = []
for r in synth_review_list: 
    history.append(r)
    completion = model.create_chat_completion(
        # model="local-model", # this field is currently unused
        messages=history,
        temperature=0.7,
        # stream=True,
        response_format={
            "type": "json_object",
            "schema": {
                "type": "object",
                "properties": {
                    "summary": {"type": "string"},
                    "sentiment": {"type": "string"}
                },
                "required": ["summary", "sentiment"],
            },
        }
    )
    answer = json.loads(completion['choices'][0]['message']['content']) # gets json in string format, then converts to json
    for k,v in answer.items():
        answers.append([[k,v]])
    history.pop()
for a in answers:
    print(a)

from_string grammar:
space ::= space_1 
space_1 ::= [ ] | 
string ::= ["] string_5 ["] space 
string_3 ::= [^"\] | [\] string_4 
string_4 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_5 ::= string_3 string_5 | 
root ::= [{] space ["] [s] [u] [m] [m] [a] [r] [y] ["] space [:] space string [,] space ["] [s] [e] [n] [t] [i] [m] [e] [n] [t] ["] space [:] space string [}] space 


llama_print_timings:        load time =   20974.79 ms
llama_print_timings:      sample time =     895.94 ms /    52 runs   (   17.23 ms per token,    58.04 tokens per second)
llama_print_timings: prompt eval time =   20974.12 ms /   177 tokens (  118.50 ms per token,     8.44 tokens per second)
llama_print_timings:        eval time =   17486.81 ms /    51 runs   (  342.88 ms per token,     2.92 tokens per second)
llama_print_timings:       total time =   39872.02 ms /   228 tokens
from_string grammar:
space ::= space_1 
space_1 ::= [ ] | 
string ::= ["] string_5 ["] space 
string_3 :

[['summary', 'The user is highly satisfied with the finance app, praising its intuitive design, user-friendliness, and the variety of features it offers for managing finances effectively.']]
[['sentiment', 'Positive']]
[['summary', 'User praises the convenience and seamless integration of the banking app across devices. They also had a positive experience with customer service resolving an issue promptly.']]
[['sentiment', 'Positive']]
[['summary', "The user expresses their satisfaction with the app's convenience and functionality, mentioning the ability to deposit checks, transfer money, and apply for loans."]]
[['sentiment', 'Positive']]
[['summary', 'The user finds the app functional for basic transactions but feels the interface is outdated and has experienced some glitches.']]
[['sentiment', 'Neutral']]
[['summary', 'The user finds the app functional but suggests improvements for customization and personalization features.']]
[['sentiment', 'Neutral']]
[['summary', 'The user menti

In [19]:
gxs.head()

,id,bank,review
0,1,GXS,Great banking app with attractive interest rat...
1,2,GXS,"A bank like no other, no bank have such amazin..."
2,3,GXS,Notice that the drop in interest rate of 0.8% ...
3,4,GXS,Sending money into my GXS account is a breeze ...
4,5,GXS,I have to say that the UI/UX is one of the bes...


In [7]:
trust.head()

,id,bank,review
132,133,Trust,"I had my card locked the night before, and con..."
133,134,Trust,"Been using it for 3 months, everything’s great..."
134,135,Trust,"On 04Sept, i accidentally clicked “yes” to be ..."
135,136,Trust,The interface is not intuitive at all. You lit...
136,137,Trust,"Hi, I’ve been using trust bank for a while now..."


In [8]:
maribank.head()

,id,bank,review
675,676,MariBank,I liked the idea where its a online bank witho...
676,677,MariBank,The onboarding process was really quick and st...
677,678,MariBank,I have to admit that the process to register i...
678,679,MariBank,Finally a mobile banking app that doesn’t lag ...
679,680,MariBank,"Overall good and smooth experience, enjoy the ..."


In [31]:
recommendations_sys_prompt = f"""You are a helpful assistant to a customer experience team that outputs in JSON. Referring to the list of mobile app reviews you will be provided, perform the following tasks: 
        1. give the topic of each review in 5 words or less.
        2. give a recommendation to the customer experience team on how to respond to the review
"""
def generate_recommendation(reviews_df,sysprompt):
    # build history
    history = [{"role": "system", "content": sysprompt},]
    answers = []
    for i in range(len(reviews_df)):
        r = reviews_df.loc[i,'review']
        history.append({"role": "user", "content": r})
        completion = model.create_chat_completion(
            # model="local-model", # this field is currently unused
            messages=history,
            temperature=0.7,
            # stream=True,
            response_format={
                "type": "json_object",
                "schema": {
                    "type": "object",
                    "properties": {
                        "topic": {"type": "string"},
                        "recomendation": {"type": "string"}
                    },
                    "required": ["topic", "recommendation"],
                },
            }
        )
        answer = json.loads(completion['choices'][0]['message']['content']) # gets json in string format, then converts to json
        for k,v in answer.items():
            answers.append([r,k,v])
        history.pop()
    # for a in answers:
    #     print(a)
    return answers

In [32]:
gxs_sample_gen = generate_recommendation(gxs.iloc[0:10,], recommendations_sys_prompt)
for g in gxs_sample_gen:
    print(g)

from_string grammar:
space ::= space_1 
space_1 ::= [ ] | 
string ::= ["] string_5 ["] space 
string_3 ::= [^"\] | [\] string_4 
string_4 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_5 ::= string_3 string_5 | 
root ::= [{] space ["] [t] [o] [p] [i] [c] ["] space [:] space string [,] space ["] [r] [e] [c] [c] [o] [m] [e] [n] [d] [a] [t] [i] [o] [n] ["] space [:] space string [}] space 

Llama.generate: prefix-match hit

llama_print_timings:        load time =   20974.79 ms
llama_print_timings:      sample time =     578.71 ms /    35 runs   (   16.53 ms per token,    60.48 tokens per second)
llama_print_timings: prompt eval time =    8872.43 ms /    71 tokens (  124.96 ms per token,     8.00 tokens per second)
llama_print_timings:        eval time =   10636.91 ms /    34 runs   (  312.85 ms per token,     3.20 tokens per second)
llama_print_timings:       total time =   20371.65 ms /   105 tokens
from_string grammar:
space ::= space_1 
space_1 ::= [ ] | 


['Great banking app with attractive interest rates! Please allow us to add and/or save payees so we don’t have to keep typing out UEN numbers or account numbers. Would be nice to be able to add the debit card to Apple Pay too!!', 'topic', 'Request for payee saving and Apple Pay integration']
['Great banking app with attractive interest rates! Please allow us to add and/or save payees so we don’t have to keep typing out UEN numbers or account numbers. Would be nice to be able to add the debit card to Apple Pay too!!', 'reccomendation', 'Consider implementing these features for improved user convenience.']
['A bank like no other, no bank have such amazing feature to separate your money into different “pockets” to keep track of your goals. Moreover the interest rate they give is the highest for the no effort! Interest are being deposit daily and it’s amazing to see 💵 coming in to the bank daily! \r\n\r\nThe user interfaces is so intuitive that every button can be found with your eyes clos